Tutorial for ensemble matching between structures based on alignments.

In [1]:
import alphaspace as al
import mdtraj
import os

For convenience, we pre aligned the Bcl-2 (PDB ID: 2XA0) and Bcl-xL (PDB ID: 3PL7) structures and prepared .pdbqt files for the protein part.

In [17]:
al.Snapshot.beta_cluster_dist = 1.3
al.Snapshot.pocket_cluster_dist = 4.7

bcl2 = mdtraj.load("bcl/bcl2.pdb")
bclxl = mdtraj.load("bcl/bclxl.pdb")
al.annotateVinaAtomTypes(receptor=bcl2,pdbqt='bcl/bcl2.pdbqt')
al.annotateVinaAtomTypes(receptor=bclxl,pdbqt='bcl/bclxl.pdbqt')

bcl2_bax = mdtraj.load("bcl/bcl2_bax.pdb")
bclxl_bax = mdtraj.load("bcl/bclxl_bax.pdb")

ss_bcl2 = al.Snapshot()
ss_bcl2.run(bcl2, bcl2_bax)

ss_bclxl = al.Snapshot()
ss_bclxl.run(bclxl, bclxl_bax)

Vina Atom Type found, calculating BScore
Vina Atom Type found, calculating BScore


In order to perform pocket comparison, we need to combine multiple snapshots into a Trajectory object.

In [18]:
bcl2_bclxl = al.Trajectory(snapshots=[ss_bcl2,ss_bclxl])

Then generate dpockets:

In [19]:
bcl2_bclxl.gen_dpockets(clust_distance=4.7)

Now we can iterate through d-pockets to see their attributes

In order to study each snapshot's contribution to the d-pocket, you can further break up d-pockets into pockets from each individual snapshots in the order they were given. Here you can access the attributes such as beta score:

In [20]:
import matplotlib.pyplot as plt
import numpy as np


for p in ss_bclxl.pockets:
    print(p.score)

# scores = []
# for dp in bcl2_bclxl.dpockets:
#     scores.append([pocket.score for pocket in dp.pockets])

# for snapshot in np.array(scores).transpose():
#     print(snapshot)

-0.678071
27.3734
-1.29674
-0.65038
6.179
-1.29674
74.3948
-0.48724
-0.939742
-0.353606
9.0958
14.627
27.8278
29.7877
9.14362
-0.0855858
27.0835
48.5416
-0.809004
34.4192
-0.0855858
-1.1482
29.5952
-0.353606
36.8949
29.2767
-0.865818
21.0386
12.288
-0.704177
33.3134
25.1707
23.314
46.5459
-1.29674
-0.353606
17.8818
51.0158
49.9316
33.1211
-0.0855858
14.891
27.0563
-0.0855858
3.36944
-0.353606
-0.353606
-0.0855858
7.89622
29.9925
20.0858
1.12626
2.17827
73.1856
24.0454
-0.42333
13.6068
21.9143
38.5913
-0.917155
9.04803
20.6994
18.2711
-0.48724
-0.48724
-0.0855858
-0.353606
5.58207
-0.917155
-1.14184
3.39724
21.6784
16.3946
-0.353606
-0.353606
6.44378
-0.353606
2.92229
52.8561
-2.95171
-0.0855858
42.7405
38.3547
59.0177
1.86158
23.1341
8.21604
64.3663
-0.353606
-0.147285
26.7349
16.9327
-0.696712
-1.14184
-0.0855858
35.8852
-0.42333
14.8695
33.085
-0.353606
-0.353606
-0.353606
-0.353606
-0.0855858
1.18521
-1.14184
-0.0855858
-0.353606
15.571
14.3067
-0.0855858
-0.809004
-0.678071
-0.0855